In [1]:
import torch
import numpy as np

## 1. Тензоры в pytorch

In [2]:
a = torch.tensor([1,2,3.0,4,5], dtype=torch.float32)
b = torch.arange(5, 10, dtype=torch.float32)
c = torch.ones(5, 4, dtype=torch.float32)
a, b, c

(tensor([1., 2., 3., 4., 5.]),
 tensor([5., 6., 7., 8., 9.]),
 tensor([[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]))

In [3]:
a.dtype

torch.float32

Над тензорами в pytorch можно производить все матиматические операции как в numpy

In [4]:
operation_names = ['a','b', 'a * b', 'a + b', 'a - b', 'a / b', 'a//b', 'a**b', 'a > b', 'a==b']
operations = [a, b, a * b, a + b, a - b, a / b, a//b, a**b, a > b, a==b]
for name, result in zip(operation_names, operations):
    print(name, result, sep='\t')

a	tensor([1., 2., 3., 4., 5.])
b	tensor([5., 6., 7., 8., 9.])
a * b	tensor([ 5., 12., 21., 32., 45.])
a + b	tensor([ 6.,  8., 10., 12., 14.])
a - b	tensor([-4., -4., -4., -4., -4.])
a / b	tensor([0.2000, 0.3333, 0.4286, 0.5000, 0.5556])
a//b	tensor([0., 0., 0., 0., 0.])
a**b	tensor([1.0000e+00, 6.4000e+01, 2.1870e+03, 6.5536e+04, 1.9531e+06])
a > b	tensor([False, False, False, False, False])
a==b	tensor([False, False, False, False, False])


Можно работать и с обычными числами, например так, используя специализированные функции pytorch для сложения, умножения, вычитания и т.д.:

In [5]:
x = 1.
y = 2.
torch.add(x, y), torch.mul(x,y), torch.sub(x,y)

(tensor(3.), tensor(2.), tensor(-1.))

Однако для простоты записи, удобнее сразу перевести все переменные в pytorch тензоры:

In [6]:
x = torch.tensor(1.)
y = torch.tensor(2.)
torch.add(x, y), torch.mul(x,y), torch.sub(x,y)

(tensor(3.), tensor(2.), tensor(-1.))

тогда вместо pytorch методов можно пользоваться переопределёнными обычными математическими операциями:

In [7]:
x + y, x * y, x - y

(tensor(3.), tensor(2.), tensor(-1.))

## 2. Автоматическое дифференцирование

In [8]:
x = torch.tensor(15., requires_grad=True) # =requires_grad=True => PyTorch будет автоматически вычислять градиенты
y = torch.tensor(24.)
f = torch.log(x**2 + y**2)

In [9]:
f

tensor(6.6859, grad_fn=<LogBackward0>)

In [10]:
# u = torch.exp(f)
# u

In [11]:
f.backward()

In [12]:
x

tensor(15., requires_grad=True)

In [13]:
x.grad

tensor(0.0375)

In [14]:
y

tensor(24.)

In [15]:
print(y.grad)

None


In [16]:
x = torch.tensor(1, dtype=torch.float32, requires_grad=True)
y = torch.tan(1 + x**2)

In [17]:
y

tensor(-2.1850, grad_fn=<TanBackward0>)

In [18]:
y.backward()

In [19]:
x.grad

tensor(11.5488)

In [20]:
import numpy as np
x = 1
2*x / np.cos(1+x**2)**2

11.548798408083835

### Более сложный пример

In [21]:
W = torch.rand(3, 4)
x = torch.arange(4, dtype=torch.float32)
W, x

(tensor([[0.6904, 0.7417, 0.6027, 0.7612],
         [0.3263, 0.9509, 0.6202, 0.9371],
         [0.1982, 0.1809, 0.3788, 0.1230]]),
 tensor([0., 1., 2., 3.]))

In [22]:
W.requires_grad_(True)

tensor([[0.6904, 0.7417, 0.6027, 0.7612],
        [0.3263, 0.9509, 0.6202, 0.9371],
        [0.1982, 0.1809, 0.3788, 0.1230]], requires_grad=True)

In [23]:
#z = W.dot(x)

Unlike NumPy’s dot, torch.dot intentionally only supports computing the dot product of two 1D tensors with the same number of elements.

In [24]:
z = W.matmul(x)
y = torch.relu(z)   # Вычисляется y, применяя функцию ReLU (Rectified Linear Unit) к z. 
                    # Это популярная нелинейная активационная функция, которая отбрасывает отрицательные значения.
target = torch.tensor([1., 1.2, 1.4])
loss = torch.sum((y - target)**2)   # Вычисляется функция потерь loss. В данном случае, это средняя квадратичная ошибка 
                                    # между y и target. То есть, (y - target)^2 и затем суммируется вся ошибка.

In [25]:
loss

tensor(24.9044, grad_fn=<SumBackward0>)

In [26]:
loss.backward() # Вызывается loss.backward(), что позволяет PyTorch автоматически вычислить градиенты параметров 
                # (в данном случае, параметры W) относительно функции потерь loss с использованием метода обратного 
                # распространения ошибки (backpropagation).

In [27]:
W.grad

tensor([[ 0.0000,  6.4615, 12.9230, 19.3845],
        [ 0.0000,  7.6048, 15.2095, 22.8143],
        [-0.0000, -0.1850, -0.3699, -0.5549]])

In [28]:
x.grad

In [29]:
W = torch.rand(3, 4, requires_grad=True)
x = torch.arange(4, dtype=torch.float32)
target = torch.tensor([1., 1.2, 1.4])
for i in range(10):
    z = W.matmul(x)
    y = torch.relu(z)
    loss = torch.sum((y - target)**2)
    loss.backward()
    with torch.no_grad():   # параметры W обновляются в соответствии с градиентами с использованием стохастического 
         W -= 0.1 * W.grad  # градиентного спуска (SGD). Обновление происходит в блоке with torch.no_grad(),
                            # чтобы избежать создания дополнительных вычислений градиентов.
    W.grad.zero_()          # градиенты обнуляются после каждой итерации с W.grad.zero(). Это важно, потому что PyTorch 
                            # автоматически аккумулирует градиенты, и вы хотите начать с чистого листа на каждой итерации.
    print(f'iteration {i}, loss {loss}')
    #print(f'grad {W.grad}')

iteration 0, loss 33.65700149536133
iteration 1, loss 4.400000095367432
iteration 2, loss 4.400000095367432
iteration 3, loss 4.400000095367432
iteration 4, loss 4.400000095367432
iteration 5, loss 4.400000095367432
iteration 6, loss 4.400000095367432
iteration 7, loss 4.400000095367432
iteration 8, loss 4.400000095367432
iteration 9, loss 4.400000095367432


In [30]:
'''Этот код выполняет градиентный спуск для минимизации функции потерь f(x). 
В каждой итерации значение x обновляется в направлении, которое уменьшает 
значение функции потерь, и градиенты пересчитываются. Этот процесс повторяется 
100 раз, и x сходится к значению, которое минимизирует функцию потерь.'''

def f(x):
    return (x - 1.5)**2

x = torch.tensor(4, dtype=torch.float32, requires_grad=True)
for i in range(100):
    y = f(x)
    y.backward()
    print(f'iteration {i}, x {x}, loss {y}, grad {x.grad}')
    with torch.no_grad():
        x -= 0.1 * x.grad
    x.grad.zero_()
    #print(f'grad {x.grad}')

iteration 0, x 4.0, loss 6.25, grad 5.0
iteration 1, x 3.5, loss 4.0, grad 4.0
iteration 2, x 3.0999999046325684, loss 2.559999704360962, grad 3.1999998092651367
iteration 3, x 2.7799999713897705, loss 1.6383999586105347, grad 2.559999942779541
iteration 4, x 2.5239999294281006, loss 1.0485758781433105, grad 2.047999858856201
iteration 5, x 2.319200038909912, loss 0.671088695526123, grad 1.6384000778198242
iteration 6, x 2.155359983444214, loss 0.429496705532074, grad 1.3107199668884277
iteration 7, x 2.0242879390716553, loss 0.2748778462409973, grad 1.0485758781433105
iteration 8, x 1.9194303750991821, loss 0.17592184245586395, grad 0.8388607501983643
iteration 9, x 1.8355443477630615, loss 0.1125900074839592, grad 0.671088695526123
iteration 10, x 1.7684354782104492, loss 0.07205760478973389, grad 0.5368709564208984
iteration 11, x 1.7147483825683594, loss 0.046116866171360016, grad 0.42949676513671875
iteration 12, x 1.6717987060546875, loss 0.029514795169234276, grad 0.343597412109

In [31]:
import torch

def f(x):
    return (x - 1.5)**2

x = torch.tensor([4], dtype=torch.float32, requires_grad=True)

optimizer = torch.optim.SGD([x], lr=0.1) # SGD с коэффициентом скорости обучения 0.1


for i in range(100):
    y = f(x)
    optimizer.zero_grad() # сбрасываем градиенты перед вычислением новых
    y.backward()          # вычисляем градиенты функции потерь f(x) относительно x
    optimizer.step()      # выполняем шаг оптимизации, обновив значение x в направлении, которое уменьшает значение f(x)
    print(f'iteration {i}, x {x}, loss {y}, grad {x.grad}')
    #     with torch.no_grad():
    #         x -= 0.1 * x.grad
    #x.grad.zero_()
    #print(f'grad {x.grad}')

iteration 0, x tensor([3.5000], requires_grad=True), loss tensor([6.2500], grad_fn=<PowBackward0>), grad tensor([5.])
iteration 1, x tensor([3.1000], requires_grad=True), loss tensor([4.], grad_fn=<PowBackward0>), grad tensor([4.])
iteration 2, x tensor([2.7800], requires_grad=True), loss tensor([2.5600], grad_fn=<PowBackward0>), grad tensor([3.2000])
iteration 3, x tensor([2.5240], requires_grad=True), loss tensor([1.6384], grad_fn=<PowBackward0>), grad tensor([2.5600])
iteration 4, x tensor([2.3192], requires_grad=True), loss tensor([1.0486], grad_fn=<PowBackward0>), grad tensor([2.0480])
iteration 5, x tensor([2.1554], requires_grad=True), loss tensor([0.6711], grad_fn=<PowBackward0>), grad tensor([1.6384])
iteration 6, x tensor([2.0243], requires_grad=True), loss tensor([0.4295], grad_fn=<PowBackward0>), grad tensor([1.3107])
iteration 7, x tensor([1.9194], requires_grad=True), loss tensor([0.2749], grad_fn=<PowBackward0>), grad tensor([1.0486])
iteration 8, x tensor([1.8355], requ

iteration 79, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 80, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 81, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 82, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 83, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 84, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 85, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], grad_fn=<PowBackward0>), grad tensor([4.7684e-07])
iteration 86, x tensor([1.5000], requires_grad=True), loss tensor([5.6843e-14], gra

In [32]:
'''В предоставленном коде происходит оптимизация параметра x для минимизации 
функции потерь f(x) = (x - 1.5)^2 вручную, без использования оптимизатора PyTorch.'''

x = torch.tensor(4, dtype=torch.float32, requires_grad=True)
y = f(x)
y.backward()
print(f'iteration {i}, x {x}, loss {y}, grad {x.grad}')
#with torch.no_grad():
x = x - 0.1 * x.grad
x.requires_grad_(True)

iteration 99, x 4.0, loss 6.25, grad 5.0


tensor(3.5000, grad_fn=<SubBackward0>)

In [33]:
x

tensor(3.5000, grad_fn=<SubBackward0>)